In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-fhvojb9x
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-fhvojb9x
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=d7dbdc4210d2b2a3480a16d03adaccf8b47308af00473c9b67f722e02d85ef6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ygnr71_/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile ass1.cu

Writing ass1.cu


In [ ]:
%%writefile ass1.cu

#include <iostream>
#include <vector>
#include <queue>
#include <omp.h>

using namespace std;

class Graph {
    int V; // Number of vertices
    vector<vector<int>> adj; // Adjacency list
public:
    Graph(int V) : V(V), adj(V) {}
    // Add an edge to the graph
    void addEdge(int v, int w) {
        adj[v].push_back(w);
    }
    // Parallel Depth-First Search
    void parallelDFS(int startVertex) {
        vector<bool> visited(V, false);
        double startTime = omp_get_wtime();
        parallelDFSUtil(startVertex, visited);
        double endTime = omp_get_wtime();
        cout << "\nExecution Time (DFS): " << endTime - startTime << " seconds" << endl;
    }
    // Parallel DFS utility function
    void parallelDFSUtil(int v, vector<bool>& visited) {
        visited[v] = true;
        cout << v << " ";
        #pragma omp parallel for
        for (int i = 0; i < adj[v].size(); ++i) {
            int n = adj[v][i];
            if (!visited[n])
                parallelDFSUtil(n, visited);
        }
    }
    // Parallel Breadth-First Search
    void parallelBFS(int startVertex) {
        vector<bool> visited(V, false);
        queue<int> q;
        double startTime = omp_get_wtime();
        visited[startVertex] = true;
        q.push(startVertex);
        while (!q.empty()) {
            int v = q.front();
            q.pop();
            cout << v << " ";
            #pragma omp parallel for
            for (int i = 0; i < adj[v].size(); ++i) {
                int n = adj[v][i];
                if (!visited[n]) {
                    visited[n] = true;
                    q.push(n);
                }
            }
        }
        double endTime = omp_get_wtime();
        cout << "\nExecution Time (BFS): " << endTime - startTime << " seconds" << endl;
    }
};

int main() {
    // Create a graph
    Graph g(7);
    g.addEdge(0, 1);
    g.addEdge(0, 2);
    g.addEdge(1, 3);
    g.addEdge(1, 4);
    g.addEdge(2, 5);
    g.addEdge(2, 6);
    cout << "Depth-First Search (DFS): ";
    g.parallelDFS(0);
    cout << endl;
    cout << "Breadth-First Search (BFS): ";
    g.parallelBFS(0);
    cout << endl;
    return 0;
}


Overwriting ass1.cu


In [ ]:
!nvcc -o ass1 ass1.cu -Xcompiler -fopenmp
!./ass1


Depth-First Search (DFS): 0 12  3 4 5 6 
Execution Time (DFS): 0.000202003 seconds

Breadth-First Search (BFS): 0 2 1 6 5 4 3 
Execution Time (BFS): 6.18e-06 seconds

